In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import requests
import time
from tqdm.notebook import tqdm
import pandas as pd

import pymysql

In [94]:
# test slot

driver = webdriver.Chrome("./driver/chromedriver")
driver.get("https://www.kurly.com/shop/board/view.php?id=recipe&no=220")

# page = list(range(220,480))

# data_dict = {'name':[], 'foriegn_lang':[], 'diff':[]}
# datasheet = pd.DataFrame(data_dict)

header = {'User-Agent':'Mozila/5.0'}
recipe = driver.page_source
recipe_soup = BeautifulSoup(recipe, 'lxml')

C:\Users\BIT\AppData\Local\Temp\ipykernel_10028\3690054255.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("./driver/chromedriver")


In [101]:
# test_slot

str(recipe_soup.select('div.pic')[0]).split('src=\"//')[1].split('"/>')[0]




'img-cf.kurly.com/shop/data/board/recipe/r/8f60c1553d2bd005'

In [65]:
recipe_context = recipe_soup.select("div.goods_recipe")[0].get_text().split("Kurly’s Recipe")[1]

title = recipe_context.split("재료")[0]
ingre = recipe_context.split("재료")[1].split("RECIPE")[0]
info = recipe_context.split("RECIPE")[1]

title = title.replace("\n","")
ingre = ingre.replace("\n","")

while info[0]=="\n":
    info = info[1:]
while info[-1]=="\n":
    info = info[:-1]
    
recipe_allinfo = [title, ingre, info]
recipe_allinfo

['어란 파스타',
 '소금 약간, 파스타 면 50~80g, 올리브오일 50ml, 마늘 2개, 후추 적당량, 어란 20g, 이탈리안 파슬리 10g, 라임즙 적당량',
 '1. 끓는 물(분량 외)에 소금을 넣고 파스타 면을 넣어 10분 정도 삶는다.\n2. 팬에 올리브오일을 두른 뒤, 다진 마늘을 넣고 볶는다.\n3. ②에 삶은 파스타 면을 넣고 섞다가 소금과 후추로 간을 한다. \n4. 접시에 ③을 담고 얇게 저민 어란을 올린다. \n5. 다진 이탈리안 파슬리를 뿌린 후 기호에 따라 라임즙을 뿌린다.']

## 마켓컬리

In [11]:
driver = webdriver.Chrome("./driver/chromedriver")

# data_dict = {'name':[], 'ingredient':[], 'information':[], 'link':[], 'img_link':[]}
# datasheet = pd.DataFrame(data_dict)

path = 'https://www.kurly.com/shop/board/list.php?id=recipe&page=29'
driver.get(path)

for i in tqdm(range(345,413)):
    page = (i//12)
    col = (i%2) +1
    row = ((i//2) % 6) +1
    
    xpath = '//*[@id="recipe_table"]/tbody/tr['+str(row)+']/td['+str(col)+']/div/ul/li'
    driver.find_element(By.XPATH,xpath).click()
    
    time.sleep(10)

    recipe = driver.page_source
    recipe_soup = BeautifulSoup(recipe, 'lxml')
    
    try:
        tmp=recipe_soup.select("div.goods_recipe")[0]
        recipe_context = tmp.get_text().split("Kurly’s Recipe")[1]
        
        title = recipe_context.split("재료")[0]
        ingre = recipe_context.split("재료")[1].split("RECIPE")[0]
        info = recipe_context.split("RECIPE")[1]
        
        title = title.replace("\n","")
        ingre = ingre.replace("\n","")
        while info[0]=="\n":
            info = info[1:]
        while info[-1]=="\n":
            info = info[:-1]
        
        link = driver.current_url
        img_link = str(recipe_soup.select('div.pic')[0]).split('src=\"//')[1].split('"/>')[0]
        datasheet.loc[i] = [title, ingre, info, link, img_link]
        
    except:
        time.sleep(1)
        
    time.sleep(5)
    
    driver.back()
    time.sleep(1)
    
    if i%12 == 11:
        next_btn = '//*[@id="recipe"]/table/tbody/tr/td/form/div/div/a['+ str((page%10) +3) +']'
        driver.find_element(By.XPATH,next_btn).click()
    
#     try:
#         if i == 0 : driver.find_element(By.XPATH,'//*[@id="content"]/div[2]/div/ul/li/a[2]').click()
#         else : driver.find_element(By.XPATH,'//*[@id="content"]/div[2]/div/ul/li[2]/a[2]').click()
#     except:
#         break
    
    time.sleep(5)

C:\Users\BIT\AppData\Local\Temp\ipykernel_3656\1211494647.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("./driver/chromedriver")


  0%|          | 0/68 [00:00<?, ?it/s]

In [2]:
datasheet.to_csv("kirly_recipe_test6.csv")

NameError: name 'datasheet' is not defined

In [5]:
company_code = 1

conn,cur= None, None
row='none'
conn = pymysql.connect(host='127.0.0.1', user='ican1', password='1234', db='ssag', charset='utf8')
cur = conn.cursor()

for i in tqdm(range(len(datasheet))):
    try:
        sql = "INSERT INTO cook (name, companycode, howtomake, link, imglink) "
        sql+= "VALUES "
        sql+= '(\"'+str(datasheet.loc[i][0])+'\",'+str(company_code)+',\"'+str(datasheet.loc[i][2])+'\",\"'
        sql+= str(datasheet.loc[i][3])+'\",\"'+str(datasheet.loc[i][4])+'\");'
#         print(sql)

        cur.execute(sql)
        conn.commit()
    except:
        print(i)
        continue
        
    finally:
        time.sleep(0.3)
conn.close()

  0%|          | 0/195 [00:00<?, ?it/s]

89
109


In [83]:
datasheet = pd.read_csv("data/kirly_recipe_test5.csv")

In [84]:
datasheet.columns

Index(['Unnamed: 0', 'name', 'ingredient', 'information', 'link', 'img_link'], dtype='object')

In [85]:
datasheet.drop('information',axis=1,inplace=True)
datasheet.drop('link',axis=1,inplace=True)
datasheet.drop('img_link',axis=1,inplace=True)
datasheet.columns

Index(['Unnamed: 0', 'name', 'ingredient'], dtype='object')

In [86]:
datasheet['ingredient'][1]

'닭 다리 2개, 닭 봉 2개, 닭 윙 2개, 올리브오일 2큰술, 소금 1/2작은술, 후춧가루 약간, 타바스코 소스 적당량레시피 참고사항미식가를 위한 큐레이션 뉴스레터<The Epicure>에 소개된 레시피입니다.더 많은 이야기가 궁금하다면 <The Epicure>를 구독하세요.지금 구독하러 가기'

In [87]:
import re
braket_open = ('(','[','<','{')
braket_close = (')',']','>','}')
ingre_data = pd.DataFrame({'index':[],'name':[],'serve':[],'ingredient':[],'quantity':[],'unit':[]})


for i in range(len(datasheet)):
    
    index = datasheet.loc[i,'Unnamed: 0']
    name = datasheet.loc[i, 'name']
    
    if '레시피 참고사항' in datasheet.loc[i,'ingredient']:
        ingre_full = datasheet.loc[i,'ingredient'].split('레시피 참고사항')[0]
    else: ingre_full = datasheet.loc[i,'ingredient']
        
    if 'Tip' in ingre_full:
        ingre_full = ingre_full.split('Tip')[0]
    
    if ingre_full.startswith(braket_open):
        serve_tag = re.split('\(|\[|\<|\{',ingre_full)[1]
        serve_tag = re.split('\)|\]|\>|\}',serve_tag)[0]
        if '인' in serve_tag:
            serve = int(serve_tag.split('인')[0][-1])
        for j in ingre_full:
            if j in braket_close:
                removal_end = ingre_full.index(j)
                break
        ingre_full = ingre_full[removal_end+1:]
        if ingre_full[0]==' ':
            ingre_full = ingre_full[1:]   
    else: serve = 1
        
    print(ingre_full)
    
    ingre_list = ingre_full.split(', ')
    split_point = False
    for j in ingre_list:
        split_point = False
        quantity = None
        unit = ""
        
        for k in j:
            if k.isdigit():
                split_point=j.index(k)
                break
        
        if split_point is not False:
            ingredient = j[:split_point]
            if ingredient != "" and ingredient[0]==' ':
                ingredient = ingredient[1:]
            if ingredient != "" and ingredient[-1]==' ':
                ingredient = ingredient[:-1]
                
            quantity_full = j[split_point:]
            

            for k in quantity_full:
                if k.isdigit() or k in ('/','~','.','-'):
                    continue
                else:
                    quantity_split_point = quantity_full.index(k)
                    break
            quantity = quantity_full[:quantity_split_point]
            unit = quantity_full[quantity_split_point:]
        
        else:
            ingredient=j
            quantity='n'
            unit='n'
        
        print(index,'\t', name,'\t', serve,'\t', ingredient,'\t', quantity,'\t', unit )

#     print('index =', datasheet.loc[i,'Unnamed: 0'],'/' , serve,'인 기준, ',ingre)

미니당근 3개, 셀러리 20cm, 샬롯 4개, 닭 몸통 부위, 물 1.5리터, 소금 약간, 찐 귀리 또는 율무, 후춧가루 약간
0 	 치킨 야채 수프 	 1 	 미니당근 	 3 	 개
0 	 치킨 야채 수프 	 1 	 셀러리 	 20 	 cm
0 	 치킨 야채 수프 	 1 	 샬롯 	 4 	 개
0 	 치킨 야채 수프 	 1 	 닭 몸통 부위 	 n 	 n
0 	 치킨 야채 수프 	 1 	 물 	 1.5 	 리터
0 	 치킨 야채 수프 	 1 	 소금 약간 	 n 	 n
0 	 치킨 야채 수프 	 1 	 찐 귀리 또는 율무 	 n 	 n
0 	 치킨 야채 수프 	 1 	 후춧가루 약간 	 n 	 n
닭 다리 2개, 닭 봉 2개, 닭 윙 2개, 올리브오일 2큰술, 소금 1/2작은술, 후춧가루 약간, 타바스코 소스 적당량
1 	 로스티드 닭 다리&날개 	 1 	 닭 다리 	 2 	 개
1 	 로스티드 닭 다리&날개 	 1 	 닭 봉 	 2 	 개
1 	 로스티드 닭 다리&날개 	 1 	 닭 윙 	 2 	 개
1 	 로스티드 닭 다리&날개 	 1 	 올리브오일 	 2 	 큰술
1 	 로스티드 닭 다리&날개 	 1 	 소금 	 1/2 	 작은술
1 	 로스티드 닭 다리&날개 	 1 	 후춧가루 약간 	 n 	 n
1 	 로스티드 닭 다리&날개 	 1 	 타바스코 소스 적당량 	 n 	 n
손질한 닭 껍질, 소금 약간, 파프리카 파우더 적당량, 라임 1/4개, 고수 2줄기
2 	 닭 껍질 튀김 	 1 	 손질한 닭 껍질 	 n 	 n
2 	 닭 껍질 튀김 	 1 	 소금 약간 	 n 	 n
2 	 닭 껍질 튀김 	 1 	 파프리카 파우더 적당량 	 n 	 n
2 	 닭 껍질 튀김 	 1 	 라임 	 1/4 	 개
2 	 닭 껍질 튀김 	 1 	 고수 	 2 	 줄기
우유 1L, 연유 200mL, 방울토마토 500g, 애플민트 2줄기, 후춧가루 약간
3 	 토마토 빙수 	 4 	 우유 	 1 	 L
3 	 토마토 빙수 	 4 	 연유 	 200 	 mL
3 	 토마토 빙

33 	 나폴리탄 파스타 	 2 	 파프리카 	 1/2 	 개
33 	 나폴리탄 파스타 	 2 	 베이컨 	 4 	 개(약 80g)
33 	 나폴리탄 파스타 	 2 	 비엔나 소시지 	 8 	 개
33 	 나폴리탄 파스타 	 2 	 파스타면 	 200 	 g
33 	 나폴리탄 파스타 	 2 	 올리브 오일 	 4 	 큰술
33 	 나폴리탄 파스타 	 2 	 토마토 케찹 	 10 	 큰술
33 	 나폴리탄 파스타 	 2 	 후춧가루·파마산치즈·파슬리가루 약간씩 	 n 	 n
배 2개, 황설탕 1/2컵, 물 1/2컵, 소금 1/2작은술, 시나몬 스틱 5개카나페 
34 	 시나몬 배 조림 & 카나페 	 1 	 배 	 2 	 개
34 	 시나몬 배 조림 & 카나페 	 1 	 황설탕 	 1/2 	 컵
34 	 시나몬 배 조림 & 카나페 	 1 	 물 	 1/2 	 컵
34 	 시나몬 배 조림 & 카나페 	 1 	 소금 	 1/2 	 작은술
34 	 시나몬 배 조림 & 카나페 	 1 	 시나몬 스틱 	 5 	 개카나페 
그린파파야 1개, 당근 1/2개, 방울토마토 10개, 땅콩 1큰술, 마늘 8개, 베트남고추(매운맛 고추) 4개, 건새우 2큰술, 라임 3개, 레몬 3개, 피쉬소스 6큰술, 황설탕 6큰술, 고수 4줄기
35 	 쏨땀 	 4 	 그린파파야 	 1 	 개
35 	 쏨땀 	 4 	 당근 	 1/2 	 개
35 	 쏨땀 	 4 	 방울토마토 	 10 	 개
35 	 쏨땀 	 4 	 땅콩 	 1 	 큰술
35 	 쏨땀 	 4 	 마늘 	 8 	 개
35 	 쏨땀 	 4 	 베트남고추(매운맛 고추) 	 4 	 개
35 	 쏨땀 	 4 	 건새우 	 2 	 큰술
35 	 쏨땀 	 4 	 라임 	 3 	 개
35 	 쏨땀 	 4 	 레몬 	 3 	 개
35 	 쏨땀 	 4 	 피쉬소스 	 6 	 큰술
35 	 쏨땀 	 4 	 황설탕 	 6 	 큰술
35 	 쏨땀 	 4 	 고수 	 4 	 줄기
감자 1개, 대파 1대, 샬롯 2개, 마늘 2개, 올리브유 2큰술, 버터 1큰

60 	 시금치 페스토 파스타 	 1 	 리가토니 	 120 	 g
60 	 시금치 페스토 파스타 	 1 	 방울토마토 	 4 	 개
60 	 시금치 페스토 파스타 	 1 	 부라타 치즈 	 120 	 g
자른 황태포 80g, 고추장 2T, 고춧가루 1T, 올리고당 2T, 설탕 1T, 진간장 1과 1/2T, 들기름 3T, 후춧가루 1/2t, 맛술 또는 청주 1T, 다진마늘 1T
61 	 황태양념구이 	 1 	 자른 황태포 	 80 	 g
61 	 황태양념구이 	 1 	 고추장 	 2 	 T
61 	 황태양념구이 	 1 	 고춧가루 	 1 	 T
61 	 황태양념구이 	 1 	 올리고당 	 2 	 T
61 	 황태양념구이 	 1 	 설탕 	 1 	 T
61 	 황태양념구이 	 1 	 진간장 	 1 	 과 1/2T
61 	 황태양념구이 	 1 	 들기름 	 3 	 T
61 	 황태양념구이 	 1 	 후춧가루 	 1/2 	 t
61 	 황태양념구이 	 1 	 맛술 또는 청주 	 1 	 T
61 	 황태양념구이 	 1 	 다진마늘 	 1 	 T
통 도다리 1마리, 전분가루 1/2컵, 식용유 3큰술, 홍고추 1/2개, 청고추 1/2개, 대파 흰 부분 1/2대, 다진 마늘 1큰술, 간장 1큰술, 설탕 1큰술, 고춧가루 1큰술, 물 3큰술
62 	 도다리 양념구이 	 1 	 통 도다리 	 1 	 마리
62 	 도다리 양념구이 	 1 	 전분가루 	 1/2 	 컵
62 	 도다리 양념구이 	 1 	 식용유 	 3 	 큰술
62 	 도다리 양념구이 	 1 	 홍고추 	 1/2 	 개
62 	 도다리 양념구이 	 1 	 청고추 	 1/2 	 개
62 	 도다리 양념구이 	 1 	 대파 흰 부분 	 1/2 	 대
62 	 도다리 양념구이 	 1 	 다진 마늘 	 1 	 큰술
62 	 도다리 양념구이 	 1 	 간장 	 1 	 큰술
62 	 도다리 양념구이 	 1 	 설탕 	 1 	 큰술
62 	 도다리 양념구이 	 1 	 고춧가루 	 1 	 큰술
62 	 도다리 양념구이 	 1 

92 	 마끼세트 	 4 	 오이 	 1 	 개
92 	 마끼세트 	 4 	 무순 	 15 	 g
랍스터 테일 2미, 소금 약간, 후추 약간, 깐마늘 2알, 이탈리안 파슬리 2줄기, 타임 1줄기, 레몬 1개, 버터 2큰술, 루토사 플레인 매쉬드포테이토 1/2컵, 감자튀김, 브로콜리 1/2개
93 	 랍스터 스테이크 	 1 	 랍스터 테일 	 2 	 미
93 	 랍스터 스테이크 	 1 	 소금 약간 	 n 	 n
93 	 랍스터 스테이크 	 1 	 후추 약간 	 n 	 n
93 	 랍스터 스테이크 	 1 	 깐마늘 	 2 	 알
93 	 랍스터 스테이크 	 1 	 이탈리안 파슬리 	 2 	 줄기
93 	 랍스터 스테이크 	 1 	 타임 	 1 	 줄기
93 	 랍스터 스테이크 	 1 	 레몬 	 1 	 개
93 	 랍스터 스테이크 	 1 	 버터 	 2 	 큰술
93 	 랍스터 스테이크 	 1 	 루토사 플레인 매쉬드포테이토 	 1/2 	 컵
93 	 랍스터 스테이크 	 1 	 감자튀김 	 n 	 n
93 	 랍스터 스테이크 	 1 	 브로콜리 	 1/2 	 개
조개류 1kg, 깐마늘 10알, 펜넬 1/2개, 양파 1/2개, 올리브오일 1/4컵, 타임 5줄기, 후춧가루 약간, 화이트와인 2컵, 토마토홀 6컵, 새우 10개, 손질 아귀 200g, 이탈리안 파슬리 2줄기
94 	 토마토 해산물 스튜 	 1 	 조개류 	 1 	 kg
94 	 토마토 해산물 스튜 	 1 	 깐마늘 	 10 	 알
94 	 토마토 해산물 스튜 	 1 	 펜넬 	 1/2 	 개
94 	 토마토 해산물 스튜 	 1 	 양파 	 1/2 	 개
94 	 토마토 해산물 스튜 	 1 	 올리브오일 	 1/4 	 컵
94 	 토마토 해산물 스튜 	 1 	 타임 	 5 	 줄기
94 	 토마토 해산물 스튜 	 1 	 후춧가루 약간 	 n 	 n
94 	 토마토 해산물 스튜 	 1 	 화이트와인 	 2 	 컵
94 	 토마토 해산물 스튜 	 1 	 토마토홀 	 6 	 컵
94 	 토마토 해산물 스튜 	

116 	 새조개 초무침 	 1 	 새조개 	 300 	 g(1팩)
116 	 새조개 초무침 	 1 	 미나리 	 10 	 줄기
116 	 새조개 초무침 	 1 	 양파 	 1/3 	 개
116 	 새조개 초무침 	 1 	 오이 	 1/3 	 개
116 	 새조개 초무침 	 1 	 사과 	 1/3 	 개
116 	 새조개 초무침 	 1 	 레몬 	 1/2 	 개
116 	 새조개 초무침 	 1 	 고추장 	 2 	 큰술
116 	 새조개 초무침 	 1 	 식초 	 2 	 큰술
116 	 새조개 초무침 	 1 	 설탕 	 2 	 큰술
116 	 새조개 초무침 	 1 	 다진 마늘 	 1 	 작은술
116 	 새조개 초무침 	 1 	 참기름 	 1 	 작은술
116 	 새조개 초무침 	 1 	 통깨 약간 	 n 	 n
알배기 배추 2통, 깻잎 2팩, 청경채 2포기, 표고버섯 3~4개, 느타리버섯 한 줌, 무 1/4개, 설도 2팩, 궁물소리 표고다시맛 2팩, 쯔유 3큰술, 유자 1개, 국수
117 	 밀푀유 나베 	 1 	 알배기 배추 	 2 	 통
117 	 밀푀유 나베 	 1 	 깻잎 	 2 	 팩
117 	 밀푀유 나베 	 1 	 청경채 	 2 	 포기
117 	 밀푀유 나베 	 1 	 표고버섯 	 3~4 	 개
117 	 밀푀유 나베 	 1 	 느타리버섯 한 줌 	 n 	 n
117 	 밀푀유 나베 	 1 	 무 	 1/4 	 개
117 	 밀푀유 나베 	 1 	 설도 	 2 	 팩
117 	 밀푀유 나베 	 1 	 궁물소리 표고다시맛 	 2 	 팩
117 	 밀푀유 나베 	 1 	 쯔유 	 3 	 큰술
117 	 밀푀유 나베 	 1 	 유자 	 1 	 개
117 	 밀푀유 나베 	 1 	 국수 	 n 	 n
감자 4개, 명란 1개, 참기름 4큰술, 파슬리가루 적당량
118 	 명란 감자볶음 	 1 	 감자 	 4 	 개
118 	 명란 감자볶음 	 1 	 명란 	 1 	 개
118 	 명란 감자볶음 	 1 	 참기름 	 4 	 큰술
118 	 명란 감자볶음

147 	 파스닙 컬리플라워 스프 	 1 	 샬롯 	 2 	 개
147 	 파스닙 컬리플라워 스프 	 1 	 마늘 	 3 	 개
147 	 파스닙 컬리플라워 스프 	 1 	 타임 	 3 	 줄기
147 	 파스닙 컬리플라워 스프 	 1 	 올리브오일 	 3 	 T
147 	 파스닙 컬리플라워 스프 	 1 	 큐민파우더 	 1/2 	 t
147 	 파스닙 컬리플라워 스프 	 1 	 파프리카 파우더 	 1/2 	 t
147 	 파스닙 컬리플라워 스프 	 1 	 치킨스톡 	 3 	 C
147 	 파스닙 컬리플라워 스프 	 1 	 샤워크림 	 1/4 	 C
147 	 파스닙 컬리플라워 스프 	 1 	 소금과 후추 약간 	 n 	 n
릭 1/2대, 계란 4개, 감자 1개, 베이컨 3줄, 생크림 1/2C, 올리브오일 1T 
148 	 릭 프리타타 	 2 	 릭 	 1/2 	 대
148 	 릭 프리타타 	 2 	 계란 	 4 	 개
148 	 릭 프리타타 	 2 	 감자 	 1 	 개
148 	 릭 프리타타 	 2 	 베이컨 	 3 	 줄
148 	 릭 프리타타 	 2 	 생크림 	 1/2 	 C
148 	 릭 프리타타 	 2 	 올리브오일 	 1 	 T 
차요테 1개, 할라피뇨 2개, 건고추 8개, 간장 200ml, 설탕 200ml, 식초 200ml, 물 200ml
149 	 차요테 장아찌 	 1 	 차요테 	 1 	 개
149 	 차요테 장아찌 	 1 	 할라피뇨 	 2 	 개
149 	 차요테 장아찌 	 1 	 건고추 	 8 	 개
149 	 차요테 장아찌 	 1 	 간장 	 200 	 ml
149 	 차요테 장아찌 	 1 	 설탕 	 200 	 ml
149 	 차요테 장아찌 	 1 	 식초 	 200 	 ml
149 	 차요테 장아찌 	 1 	 물 	 200 	 ml
로즈마리 잎 1~2줄기분, 돼지 등심 한덩이(뼈 바른다) 1kg, 버터 25g, 올리브기름 6큰술, 다진마늘, 다진양파, 달지 않은 화이트 와인 175ml, 화이트 와인, 식초 1큰술, 디종 머스터드 

163 	 비건 볼로네제 스파게티 	 1 	 애호박 	 1/2 	 개
163 	 비건 볼로네제 스파게티 	 1 	 가지 	 1/2 	 개
163 	 비건 볼로네제 스파게티 	 1 	 마늘 	 2 	 톨
163 	 비건 볼로네제 스파게티 	 1 	 오레가노 약간 	 n 	 n
163 	 비건 볼로네제 스파게티 	 1 	 소금 약간 	 n 	 n
163 	 비건 볼로네제 스파게티 	 1 	 후추 약간 	 n 	 n
보타르가, 어린잎채소 100g, 래디쉬 1개, 계란 1개, 엑스트라 버진 올리브유 1/2c, 레몬 2개(50g), 소금, 후추, 머스터드 1t
164 	 수란을 얹은 보타르가 어린잎 샐러드 	 1 	 보타르가 	 n 	 n
164 	 수란을 얹은 보타르가 어린잎 샐러드 	 1 	 어린잎채소 	 100 	 g
164 	 수란을 얹은 보타르가 어린잎 샐러드 	 1 	 래디쉬 	 1 	 개
164 	 수란을 얹은 보타르가 어린잎 샐러드 	 1 	 계란 	 1 	 개
164 	 수란을 얹은 보타르가 어린잎 샐러드 	 1 	 엑스트라 버진 올리브유 	 1/2 	 c
164 	 수란을 얹은 보타르가 어린잎 샐러드 	 1 	 레몬 	 2 	 개(50g)
164 	 수란을 얹은 보타르가 어린잎 샐러드 	 1 	 소금 	 n 	 n
164 	 수란을 얹은 보타르가 어린잎 샐러드 	 1 	 후추 	 n 	 n
164 	 수란을 얹은 보타르가 어린잎 샐러드 	 1 	 머스터드 	 1 	 t
양고기 모듬육 540g, 카놀라 오일 2작은술, 당근 2개, 셀러리 2개, 토마토 홀 통조림 1개 (425g), 키드니 빈스(적강낭콩) 통조림 1개 (310g), 비프 스톡 1컵, 레드와인 1/2컵, 매시드 포테이토
165 	 램 케서롤 	 4 	 양고기 모듬육 	 540 	 g
165 	 램 케서롤 	 4 	 카놀라 오일 	 2 	 작은술
165 	 램 케서롤 	 4 	 당근 	 2 	 개
165 	 램 케서롤 	 4 	 셀러리 	 2 	 개
165 	 램 케서롤 	 4 	 토마토 홀

401 	 삼겹살을 넣은 반미 샌드위치 	 3 	 굴소스 	 1 	 큰술
401 	 삼겹살을 넣은 반미 샌드위치 	 3 	 설탕 	 4 	 작은술
401 	 삼겹살을 넣은 반미 샌드위치 	 3 	 참기름 	 1 	 큰술
401 	 삼겹살을 넣은 반미 샌드위치 	 3 	 마늘 	 1 	 큰술
401 	 삼겹살을 넣은 반미 샌드위치 	 3 	 식초 	 3 	 큰술
401 	 삼겹살을 넣은 반미 샌드위치 	 3 	 설탕 	 3 	 큰술
401 	 삼겹살을 넣은 반미 샌드위치 	 3 	 소금 	 1 	 큰술
치커리 1/4줌, 로메인 상추 한 줌, 케일 1/4줌, 방울토마토 2-3개, 베이비 채소 1/2팩, 달걀 1개, 크루통 1큰술, 퀴노아 1큰술, 파마산 치즈, 소금, 후추, 디종 머스터드 2큰술, 식초 1큰술, 꿀 1큰술, 올리브유 1큰술
405 	 퀴노아 수란 샐러드 	 1 	 치커리 	 1/4 	 줌
405 	 퀴노아 수란 샐러드 	 1 	 로메인 상추 한 줌 	 n 	 n
405 	 퀴노아 수란 샐러드 	 1 	 케일 	 1/4 	 줌
405 	 퀴노아 수란 샐러드 	 1 	 방울토마토 	 2-3 	 개
405 	 퀴노아 수란 샐러드 	 1 	 베이비 채소 	 1/2 	 팩
405 	 퀴노아 수란 샐러드 	 1 	 달걀 	 1 	 개
405 	 퀴노아 수란 샐러드 	 1 	 크루통 	 1 	 큰술
405 	 퀴노아 수란 샐러드 	 1 	 퀴노아 	 1 	 큰술
405 	 퀴노아 수란 샐러드 	 1 	 파마산 치즈 	 n 	 n
405 	 퀴노아 수란 샐러드 	 1 	 소금 	 n 	 n
405 	 퀴노아 수란 샐러드 	 1 	 후추 	 n 	 n
405 	 퀴노아 수란 샐러드 	 1 	 디종 머스터드 	 2 	 큰술
405 	 퀴노아 수란 샐러드 	 1 	 식초 	 1 	 큰술
405 	 퀴노아 수란 샐러드 	 1 	 꿀 	 1 	 큰술
405 	 퀴노아 수란 샐러드 	 1 	 올리브유 	 1 	 큰술
